## Import required libraries

In [1]:
# Import standard libraries
import numpy as np
import pandas as pd

# Import functions for data preprocessing and preparation
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder, OrdinalEncoder, StandardScaler

# Import classification models
from xgboost import XGBClassifier
from lightgbm import LGBMClassifier
from imblearn.ensemble import BalancedRandomForestClassifier
from sklearn.ensemble import VotingClassifier, HistGradientBoostingClassifier

# Import library for hyperparameter optimization
import optuna

# Import evaluation metric
from sklearn.metrics import roc_auc_score

## Function Definitions

In [2]:
def create_features(df):
    '''
    This function takes a dataframe object and creates new features to train the model on.
    '''
    
    # Create a new feature by divided 'Air temperature' from 'Process temperature'
    df["Temperature ratio"] = df['Process temperature [K]'] / df['Air temperature [K]']
    
    # Create a new feature by multiplying 'Torque' and 'Rotational speed'
    df['Torque * Rotational speed'] = df['Torque [Nm]'] * df['Rotational speed [rpm]']

    # Create a new feature by multiplying 'Torque' by 'Tool wear'
    df['Torque * Tool wear'] = df['Torque [Nm]'] * df['Tool wear [min]']
    
    new_cols = [
        'Temperature ratio', 
        'Torque * Rotational speed',
        'Torque * Tool wear'
    ]
    
    return df, new_cols

def rename(df):
    '''
    This function takes a dataframe object, and replaces the invalid characters in the column names.
    '''
    
    df.columns = df.columns.str.replace('\[.*?\]', '', regex=True)
    df.columns = df.columns.str.strip()
    
    return df

def replace_outliers(data, num_cols):
    '''
    This function takes a dataframe object, and for each column in num_cols, 
    calculates the upper and lower limits using the interquartile range, and clips the values.
    '''
    for col in data[num_cols]:
        Q1 = data[col].quantile(0.25)
        Q3 = data[col].quantile(0.75)
        IQR = Q3 - Q1
        
        lower = Q1 - 1.5*IQR
        upper = Q3 + 1.5*IQR
        if col != 'TemperatureChangeRate':
            data[col] = np.clip(data[col], lower-100, upper+200)
        else:
            data[col] = np.clip(data[col], lower, upper+1)
    return data

## Read in data for training

In [3]:
# Read in competition training data, as well as original dataset the competition data is based on.

train_df = pd.read_csv('train.csv', index_col=0)
original = pd.read_csv('machine failure.csv', index_col=0)

target_col = 'Machine failure'
num_cols = [
    'Air temperature [K]',
    'Process temperature [K]',
    'Rotational speed [rpm]',
    'Torque [Nm]',
    'Tool wear [min]'
]
binary_cols = [
    'TWF',
    'HDF',
    'PWF',
    'OSF',
    'RNF'
]
cat_cols = train_df.select_dtypes(include=['object']).columns.tolist()

print(f"train shape :{train_df.shape}")
print(f"original shape :{original.shape}")

# Concatenate train and original dataframes, and prepare train dataset
train_df = pd.concat([train_df, original])

print(f"combined shape :{train_df.shape}")

train shape :(136429, 13)
original shape :(10000, 13)
combined shape :(146429, 13)


## Exploratory Data Analysis

In [4]:
# summary table function

pd.options.display.float_format = '{:,.2f}'.format

def summary(df):
    print(f'data shape: {df.shape}')
    
    summ = pd.DataFrame(df.dtypes, columns=['data type'])
    summ['# missing'] = df.isnull().sum().values 
    summ['% missing'] = df.isnull().sum().values / len(df) * 100
    summ['# unique'] = df.nunique().values
    
    desc = pd.DataFrame(df.describe(include='all').transpose())
    summ['min'] = desc['min'].values
    summ['max'] = desc['max'].values
    summ['average'] = desc['mean'].values
    summ['standard_deviation'] = desc['std'].values
    
    return summ

summary(train_df)

data shape: (146429, 13)


,data type,# missing,% missing,# unique,min,max,average,standard_deviation
Product ID,object,0,0.00,10000,NaN,NaN,NaN,NaN
Type,object,0,0.00,3,NaN,NaN,NaN,NaN
Air temperature [K],float64,0,0.00,96,295.30,304.50,299.87,1.87
Process temperature [K],float64,0,0.00,82,305.70,313.80,309.95,1.39
Rotational speed [rpm],int64,0,0.00,957,"1,168.00","2,886.00","1,521.59",141.95
Torque [Nm],float64,0,0.00,615,3.80,76.60,40.32,8.61
Tool wear [min],int64,0,0.00,246,0.00,253.00,104.65,63.95
Machine failure,int64,0,0.00,2,0.00,1.00,0.02,0.13
TWF,int64,0,0.00,2,0.00,1.00,0.00,0.04
HDF,int64,0,0.00,2,0.00,1.00,0.01,0.07


The predictive maintenance data set has 146,429 rows and 13 columns. 

These are the 12 feature variables:
<ul>
    <li>Product ID : an identification code for each product, a combination of letters and numbers.</li> 
    <li>Type : type of product or device (Low, Medium, and High).</li>
    <li>Air temperature [K] : air temperature in Kelvin units.</li>
    <li>Process temperature [K] : temperature in the production process in units of Kelvin.</li>
    <li>
        Rotational speed [rpm] : number of revolutions in one minute. 
        (Calculated from a power of 2.86kW, overlaid with a normally distributed noise)
    </li>
    <li>
        Torque [Nm] : the force that causes an object to rotate, measured in Newton-meters (Nm). 
        (Torque values are normally distributed around 40 Nm and no negative values)
    </li>
    <li>
        Tool wear [min] : total time required to erode or damage production tools 
        due to gradual wear and tear of cutting tools due to regular operation.
    </li>
</ul>

Then there are 5 types of variables that indicate damage to the machine:
<ul>
    <li>TWF (Tool Wear Failure) : tool failure that will be replaced at a randomly selected tool wear time between 200 - 240 minutes.</li>
    <li>HDF (Heat Dissipation Failure) : if the difference between air temperature and process temperature (temperature difference) is below 8.6 K and the rotational speed is below 1380 rpm.</li>
    <li>PWF (Power Failure) : multiplication of torque and rotational speed (rad/s) equal to the power required for the process. If the power is below 3500 W or above 9000 W then the process will fail causing machine failure.</li>
    <li>OSF (Overstrain Failure) is if the product of tool wear and torque exceeds 11,000 minNm for type L machine, 12,000 minNm for type M machine, and 13,000 minNm for type H machine, then the machine will be damaged due to overstrain.</li>
    <li>RNF (Random Failures) is every process has 0.1% chance to fail regardless of process parameters.</li>
</ul>

Finally, the target variable:
<ul>
    <li>Machine failure is a situation where the machine cannot function as expected or the machine is damaged.</li>
</ul>

<h6>Description source from original dataset</h6>

In [5]:
corr = train_df.corr(numeric_only=True)
corr.style.background_gradient(cmap='coolwarm')

,Air temperature [K],Process temperature [K],Rotational speed [rpm],Torque [Nm],Tool wear [min],Machine failure,TWF,HDF,PWF,OSF,RNF
Air temperature [K],1.000000,0.857645,0.017702,-0.007564,0.017026,0.069595,0.004853,0.104220,0.007832,0.007573,0.005641
Process temperature [K],0.857645,1.000000,0.012359,-0.007073,0.012988,0.031734,0.004956,0.043093,0.003222,0.005473,0.005573
Rotational speed [rpm],0.017702,0.012359,1.000000,-0.788496,0.004104,-0.057575,-0.002884,-0.085981,0.065107,-0.066037,-0.004226
Torque [Nm],-0.007564,-0.007073,-0.788496,1.000000,-0.003289,0.150027,0.009153,0.104958,0.053833,0.116844,0.008584
Tool wear [min],0.017026,0.012988,0.004104,-0.003289,1.000000,0.061215,0.053629,0.010658,0.005800,0.072668,-0.001257
Machine failure,0.069595,0.031734,-0.057575,0.150027,0.061215,1.000000,0.319621,0.566309,0.406044,0.500060,0.001619
TWF,0.004853,0.004956,-0.002884,0.009153,0.053629,0.319621,1.000000,0.007767,0.034194,0.036778,0.004912
HDF,0.104220,0.043093,-0.085981,0.104958,0.010658,0.566309,0.007767,1.000000,0.042090,0.064555,0.000332
PWF,0.007832,0.003222,0.065107,0.053833,0.005800,0.406044,0.034194,0.042090,1.000000,0.095089,0.000155
OSF,0.007573,0.005473,-0.066037,0.116844,0.072668,0.500060,0.036778,0.064555,0.095089,1.000000,-0.000933


## Data Preprocessing & Feature Engineering

In [ ]:
# Create new features
new_cols = []
train_df, new_cols = create_features(train_df)

# Rename columns so that the data is accepted by the classifier models
train_df = rename(train_df)

# Remove alphabetical characters from 'Product ID' column, whose values will then be able to be scaled later
train_df['Product ID'] = train_df['Product ID'].str.replace('M','').str.replace('L','').str.replace('H','').astype(int)

# Sum up total number of failures to a single column
train_df['TotalFailures'] = train_df[['TWF', 'HDF', 'PWF', 'OSF', 'RNF']].sum(axis=1)

# Drop unnecessary columns
train_df.drop(['Type', 'TWF', 'HDF', 'PWF', 'OSF', 'RNF'], axis=1, inplace=True)

numeric_cols = ['Product ID', 'Air temperature', 'Process temperature', 'Rotational speed',
                'Torque', 'Tool wear','Torque * Rotational speed','Temperature ratio']

train_df = replace_outliers(train_df, numeric_cols)

# Standardize numeric column values
sc = StandardScaler()
numeric_cols = [c for c in train_df.columns if 'failure' not in c.lower()]
train_df[numeric_cols] = sc.fit_transform(train_df[numeric_cols])

train_df = train_df.reset_index(drop=True)

In [ ]:
# Split data into features (X) and target variable (y)
X = train_df.drop(f'{target_col}', axis=1)
y = train_df[f'{target_col}']

# Split the data into train and test sets
X_train, X_val, y_train, y_val = train_test_split(X, y, test_size=0.2, random_state=42)

## Model Training, Model Evaluation & Hyperparameter Optimization

In [ ]:
## Define each model and their Optuna experiments

def xgb_objective(trial):
    params = {
        'tree_method': 'gpu_hist',
        'n_estimators': trial.suggest_int('n_estimators', 100, 400),
        'learning_rate': trial.suggest_float('learning_rate', 0.001, 0.1),
        'max_depth': trial.suggest_int('max_depth', 3, 10),
        'subsample': trial.suggest_float('subsample', 0.5, 1.0),
        'colsample_bytree': trial.suggest_float('colsample_bytree', 0.5, 1.0),
    }

    clf = XGBClassifier(**params, random_state=42, objective="binary:logistic")
    clf.fit(X_train, y_train, eval_set=[(X_val, y_val)], early_stopping_rounds=20, verbose=False)

    # Predict probabilities for the validation data
    y_pred_proba = clf.predict_proba(X_val)[:, 1]

    # Calculate ROC AUC score for validation predictions
    roc_auc = roc_auc_score(y_val, y_pred_proba)

    return roc_auc

xgb_study = optuna.create_study(direction='maximize')
xgb_study.optimize(xgb_objective, n_trials=100)

# Print the best hyperparameters and corresponding ROC AUC scores - XGBClassifier
xgb_best_params = xgb_study.best_params
xgb_best_score = xgb_study.best_value
print("XGB Best Hyperparameters: ", xgb_best_params)
print("XGB Best ROC AUC Score: ", xgb_best_score)

def lgbm_objective(trial):

    params = {
        'n_estimators': trial.suggest_int('n_estimators', 100, 900),
        'learning_rate': trial.suggest_float('learning_rate', 0.001, 0.1),
        'max_depth': trial.suggest_int('max_depth', 3, 10),
        'num_leaves': trial.suggest_int('num_leaves', 10, 1000),
        'subsample': trial.suggest_float('subsample', 0.5, 1.0),
        'colsample_bytree': trial.suggest_float('colsample_bytree', 0.5, 1.0),
    }

    clf = LGBMClassifier(**params, is_unbalance=True, random_state=42)
    clf.fit(X_train, y_train)

    # Predict probabilities for the validation data
    y_pred_proba = clf.predict_proba(X_val)[:, 1]

    # Calculate ROC AUC score for validation predictions
    roc_auc = roc_auc_score(y_val, y_pred_proba)

    return roc_auc

study = optuna.create_study(direction='maximize')
study.optimize(lgbm_objective, n_trials=100)

# Print the best hyperparameters and corresponding ROC AUC score
lgbm_best_params = study.best_params
lgbm_best_score= study.best_value
print("LGBM Best Hyperparameters: ",lgbm_best_params)
print("LGBM Best ROC AUC Score: ", lgbm_best_score)

def brc_objective(trial):
    # Define the hyperparameters to optimize
    params = {
        'n_estimators': trial.suggest_int('n_estimators', 100, 500),
        'max_depth': trial.suggest_categorical('max_depth', [3, 5, 10, None]),
        'min_samples_split': trial.suggest_int('min_samples_split', 2, 10),
        'min_samples_leaf': trial.suggest_int('min_samples_leaf', 1, 10),
        'max_features': trial.suggest_categorical('max_features', ['auto', 'sqrt', 'log2', None])
    }

    clf = BalancedRandomForestClassifier(**params, random_state=42)
    clf.fit(X_train, y_train)

    # Predict probabilities for the validation data
    y_pred_proba = clf.predict_proba(X_val)[:, 1]

    # Calculate ROC AUC score for validation predictions
    roc_auc = roc_auc_score(y_val, y_pred_proba)

    return roc_auc

study = optuna.create_study(direction='maximize')
study.optimize(brc_objective, n_trials=100)

# Print the best hyperparameters and corresponding ROC AUC score
brc_best_params = study.best_params
brc_best_score = study.best_value
print("BRC Best Hyperparameters: ", brc_best_params)
print("BRC Best ROC AUC Score: ", brc_best_score)

def hgbc_objective(trial):
    params = {
        'learning_rate': trial.suggest_float('learning_rate', 0.01, 0.2),
        'max_iter': trial.suggest_int('max_iter', 100, 900),
        'max_depth': trial.suggest_int('max_depth', 3, 10),
        'l2_regularization': trial.suggest_float('l2_regularization', 0.0, 1.0)
    }

    clf = HistGradientBoostingClassifier(**params, random_state=42)
    clf.fit(X_train, y_train)
    
    y_pred_proba = clf.predict_proba(X_val)[:, 1]
    roc_auc = roc_auc_score(y_val, y_pred_proba)
    
    return roc_auc

study = optuna.create_study(direction='maximize')
study.optimize(hgbc_objective, n_trials=100)

hgbc_best_params = study.best_params
hgbc_best_score = study.best_value

print("HGBC Best Hyperparameters:", hgbc_best_params)
print("HGBC Best ROC AUC Score:", hgbc_best_score)

In [ ]:
## Retrain all models using best parameters found from Optuna hyperparameter optimization

xgb_model = XGBClassifier(**xgb_best_params, objective="binary:logistic", random_state=42)
xgb_model.fit(X_train,y_train)
y_pred = xgb_model.predict(X_val)
xgb_best_score = roc_auc_score(y_val, xgb_model.predict_proba(X_val)[:,1])
print("XGB Best ROC AUC Score: ", xgb_best_score)

lgbm_model = LGBMClassifier(**lgbm_best_params, is_unbalance=True, random_state=42)
lgbm_model.fit(X_train,y_train)
y_pred = lgbm_model.predict(X_val)
lgbm_best_score = roc_auc_score(y_val, lgbm_model.predict_proba(X_val)[:,1])
print("LGBM Best ROC AUC Score: ", lgbm_best_score)

brc_model = BalancedRandomForestClassifier(**brc_best_params, random_state=42)
brc_model.fit(X_train, y_train)
y_pred = brc_model.predict(X_val)
brc_best_score = roc_auc_score(y_val, brc_model.predict_proba(X_val)[:,1])
print("BRC Best ROC AUC Score: ", brc_best_score)

hgbc_model = HistGradientBoostingClassifier(**hgbc_best_params, random_state=42)
hgbc_model.fit(X_train, y_train)
y_pred = hgbc_model.predict(X_val)
hgbc_best_score = roc_auc_score(y_val, hgbc_model.predict_proba(X_val)[:,1])
print("HGBC Best ROC AUC Score: ", hgbc_best_score)

# Create ensemble model of all models, and train the model once more
ensemble = VotingClassifier([('xgb', xgb_model), ('lgbm', lgbm_model), ('brc', brc_model), ('hgbc', hgbc_model)], 
                            voting='soft')

ensemble.fit(X_train, y_train)
y_pred = ensemble.predict(X_val)
ensemble_best_score = roc_auc_score(y_val, ensemble.predict_proba(X_val)[:,1])
print("Ensemble Best ROC AUC Score: ", ensemble_best_score)

## Predictions for test set

In [ ]:
## Prepare test set

test_df = pd.read_csv('test.csv', index_col=0)

# Create new features
new_cols = []
test_df, new_cols = create_features(test_df)

# Rename columns so that the data is accepted by the classifier models
test_df = rename(test_df)

# Remove alphabetical characters from 'Product ID' column, whose values will then be able to be scaled later
test_df['Product ID'] = test_df['Product ID'].str.replace('M','').str.replace('L','').str.replace('H','').astype(int)

# Sum up total number of failures to a single column
test_df['TotalFailures'] = test_df[['TWF', 'HDF', 'PWF', 'OSF', 'RNF']].sum(axis=1)

# Drop unnecessary columns
test_df.drop(['Type', 'TWF', 'HDF', 'PWF', 'OSF', 'RNF'], axis=1, inplace=True)

numeric_cols = ['Product ID', 'Air temperature', 'Process temperature', 'Rotational speed',
                'Torque', 'Tool wear','Torque * Rotational speed','Temperature ratio']

test_df = replace_outliers(test_df, numeric_cols)

# Standardize numeric column values
sc = StandardScaler()
numeric_cols = [c for c in test_df.columns if 'failure' not in c.lower()]
test_df[numeric_cols] = sc.fit_transform(test_df[numeric_cols])

ytest_pred_proba = ensemble.predict_proba(test_df)[:, 1]  # Probability of the positive class
test_df['Machine failure'] = ytest_pred_proba
test_df.reset_index()[['id', 'Machine failure']].to_csv('sub.csv', index=False)